# Setup & Bibliotheken

In [ ]:
# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

!pip install -q sentence-transformers transformers accelerate torch pandas tqdm

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")
!pip install transformers torch pandas tqdm -q

In [15]:

import platform
import os
import pandas as pd
import random
from tqdm import tqdm
import tensorflow_probability as tfp
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from google.colab import files, drive
from datetime import datetime
import time
import shutil
import re

from google.colab import drive



In [2]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
#print(df.head)

In [ ]:
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [ ]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)
erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]



#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")

FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

## Datenaugmentation mit Trainingsdaten (ca. 11.900 Rezensionen)

## Paraphrasieren

In [ ]:

# Google Drive mounten
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ABSA_Paraphrase/'
os.makedirs(SAVE_PATH, exist_ok=True)

print("Ordner & Bibliotheken geladen!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ordner & Bibliotheken geladen!


Beste Lösung bislang

In [29]:


def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Paraphrasierung

SYNONYMS = {
    "schmutzig": ["dreckig", "unhygienisch", "unrein" , "unsauber", "verschmutzt", "versifft", "gammelig", "schmuddelig"],
    "hygienisch": ["sauber", "rein", "keimfrei", "gesäubert", "picobello", "blitzeblank"],
    "arbeite": ["bin tätig", "beschäftigt", "bin angestellt"],
    "seit": ["seit etwa", "schon seit", "seit circa"],
    "im": ["in", "bei"],
    "und": ["sowie", "–", "&"],
    "meine": ["meine beiden"],
    "waren": ["lebten", "wohnten"],
    "dort": ["dort", "im Heim", "in der Einrichung"],
    "ist": ["bleibt"],
    "sehr": ["wirklich", "besonders"],
    "kollegial": ["freundschaftlich", "kameradschaftlich", "wie ein guter Kollege"],
    "betreuung": ["Pflege", "Fürsorge", "Hilfe", "Obhut", "Unterstützung"],
    "bewohner": ["Senioren", "Pflegebedürftige"],
    "essen": [ "Speisen", "Nahrung"],
    "fest": ["Feier", "Party" "Fete"],
    "beliebt": ["geschätzt", "begehrt", "wird gemocht"],
    "schlecht": ["mies", "mieserabel", "schrecklich" "mangelhaft", "unkorrekt", "daneben"],
    "warum": ["weshalb", "wieso"],
    "gut": ["angenehm"],
    "top": ["super", "toll", "einwandfrei", "hammer"],
    "sauber": ["ordentlich", "gepflegt"],
    "lecker": ["köstlich", "schmackhaft", "gut"],
    "okay": ["in Ordnung", "befriedigend"]

}# kann noch erweitert werden!

def synonym_replace(text):
    words = text.split()
    new_words = []
    for w in words:
        lower = w.lower().rstrip('.,!?')
        if lower in SYNONYMS and random.random() < 0.5:
            repl = random.choice(SYNONYMS[lower])
            if w[0].isupper(): repl = repl.capitalize()
            new_words.append(repl + w[len(lower):])
        else:
            new_words.append(w)
    return ' '.join(new_words)

def hybrid_paraphrase(text, n=3):
    original = str(text).strip()
    if len(original) < 20:
        return [original] * n

    results = set()
    max_attempts = n * 5  # max. 15 Versuche

    for _ in range(max_attempts):
        if len(results) >= n:
            break

        step1 = synonym_replace(original)
        if len(step1.split()) > 10 and random.random() < 0.3:
            sentences = re.split(r'(?<=[.!?])\s+', step1)
            if len(sentences) > 1:
                random.shuffle(sentences)
                step1 = ' '.join(sentences)
        results.add(step1)

    # garantierte Ausgabe,damit Prgramm an dieser schleife nicht hängen bleibt
    result_list = list(results)
    while len(result_list) < n:
        result_list.append(original)

    return result_list[:n]

# erfahrungen_clean.csv laden, falls noch nicht vorhanden
try:
    df
except NameError:
    df = pd.read_csv("erfahrungen_clean.csv")
    df = df.dropna(subset=['Erfahrungsbericht des Nutzers']).reset_index(drop=True)
    print(f"{len(df)} Rezensionen geladen!")

# Test mit 10 Rezensionen aus erfahrungen_clean.csv
test_indices = random.sample(range(len(df)), 10)
print(f"\n10-TEST: HYBRID \n")
print("="*90)

for i, idx in enumerate(test_indices, 1):
    text = str(df.iloc[idx]['Erfahrungsbericht des Nutzers'])
    print(f"\n{i}. ORIGINAL (Index {idx}):")
    print(text[:200] + ("..." if len(text) > 200 else ""))
    print("\n   PARAPHRASEN:")
    paras = hybrid_paraphrase(text, 3)
    print("Wir sind NACH 'paras'")
    for j, p in enumerate(paras, 1):
        print(f"   {j}: {p[:200]}{'...' if len(p) > 200 else ''}")
    print("-" * 90)





10-TEST: HYBRID 


1. ORIGINAL (Index 1893):
Freundliche, schnelle, unbürokratische Hilfe, schöne Einrichtung mit viel Aktivitäten für Senioren, guter Mittagstisch, nette Atmosphäre, ruhige Lage

   PARAPHRASEN:
Wir sind NACH 'paras'
   1: Freundliche, schnelle, unbürokratische Hilfe, schöne Einrichtung mit viel Aktivitäten für Senioren, guter Mittagstisch, nette Atmosphäre, ruhige Lage
   2: Freundliche, schnelle, unbürokratische Hilfe, schöne Einrichtung mit viel Aktivitäten für Senioren, guter Mittagstisch, nette Atmosphäre, ruhige Lage
   3: Freundliche, schnelle, unbürokratische Hilfe, schöne Einrichtung mit viel Aktivitäten für Senioren, guter Mittagstisch, nette Atmosphäre, ruhige Lage
------------------------------------------------------------------------------------------

2. ORIGINAL (Index 3213):
Wer dieses Haus schlecht redet, hat keinen Einblick wie es in anderen Einrichtungen aussieht. Es ist einfach rundum ein tolles Haus

   PARAPHRASEN:
Wir sind NACH 'paras'
   1: We

In [ ]:
# finaler Batch: Alle Rezensionen aus "efahrungen_clean.csv" augmentieren
print(f"\nSTART: Paraphrasierung aller {len(df)} Rezensionen -> {len(df)*3} Samples...")
aug_data = []

for idx in tqdm(range(len(df)), desc="Augmentiere", unit="text"):
    text = df.iloc[idx]['Erfahrungsbericht des Nutzers']
    variants = hybrid_paraphrase(text, 3)
    for v_id, para in enumerate(variants, 1):
        aug_data.append({
            'original_index': idx,
            'original': text,
            'paraphrase': para,
            'variant_id': v_id
        })

# Speichern
aug_df = pd.DataFrame(aug_data)
output_path = "/content/drive/MyDrive/ABSA_Paraphrase/augmented_erfahrungen.csv"
aug_df.to_csv(output_path, index=False)

print(f"\nFERTIG!")
print(f"→ {len(aug_df)} Paraphrasen gespeichert")
print(f"→ Datei: {output_path}")
print(f"→ Robuster Datensatz für Grok 3 Mini bei der ABSA vorbereitet!")

für Fine Tuning:
Hugging Face: Fine-tune ein Grok-ähnliches Modell (z. B. "xai-org/grok-1" oder "reedmayhew/Grok-3-gemma3-4B-distilled" als Distillation von Grok 3). Das ist open-source und GitHub-freundlich.

Code-Beispiel (Distillation von Grok 3 via API + Fine-Tuning auf Gemma-3 4B):


Best Practices (aus Reddit, GitHub Blog, HF Docs):

Lizenz: Füge eine LICENSE (z. B. Apache 2.0) hinzu, da Grok-Modelle proprietär sind – dein Fine-Tuning erbt das.
Model Card: Erstelle eine (HF-Template: https://huggingface.co/docs/hub/model-cards) mit Details zu Daten (12.000 Rezensionen), Augmentation und Metriken.
Datenschutz: Anonymisiere sensible Daten in der CSV (z. B. Namen entfernen), bevor du hochlädst – Pflegeberichte könnten personenbezogen sein (DSGVO-konform!).
Versionskontrolle: Nutze GitHub Releases für große Dateien (z. B. Weights als Asset).
Integration mit HF: Push zu Hugging Face Hub und verlinke im GitHub-Repo – HF ist GitHub-ähnlich und unterstützt Grok-Destillationen (z. B. "reedmayhew/Grok-3-gemma3-4B-distilled").



3. Empfehlung für deine Bachelorarbeit

Starte mit Option A/B: Prompt Engineering für schnelle Tests, dann Hugging Face für echtes Fine-Tuning (ca. 30–60 Min. in Colab mit GPU).
GitHub-Repo als Portfolio: Lade Code, Skripte und Metriken hoch – das zeigt Reproduzierbarkeit. Vermeide rohe Weights (>100 MB) direkt; verlinke sie.
Nächste Schritte: Hole dir einen xAI API-Key und HF-Token. Teste den Code in Colab.

Falls du Hilfe beim Setup (z. B. API-Key, spezifischer Code-Fehler) brauchst oder mehr Details zu einer Option, lass es mich wissen!

In [ ]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485